In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [2]:
# Load environment variables
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

C:\Users\urjam\AppData\Local\Temp\ipykernel_2920\1895384985.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [4]:
class Agent:
    def __init__(self, name, system_prompt, llm):
        self.name = name
        self.system_prompt = system_prompt
        self.llm = llm
        self.chat_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_prompt),
            HumanMessagePromptTemplate.from_template("{conversation_history}\n{input}")
        ])
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.chat_prompt,
            verbose=False
        )

    def respond(self, input_text, conversation_history):
        response = self.chain.predict(
            input=input_text,
            conversation_history=conversation_history
        )
        return response
        
# Define improved system prompts for each agent
agents = {
    "Product_Owner": Agent(
        name="Product_Owner",
        system_prompt=(
            """You are the Product Owner for the Renter's Insurance platform. Create detailed and concise user stories with success and acceptance criteria.
            
            **Tasks:**
            - Define **Functional Stories** (e.g., user registration, quote requests, claims management).
            - Define **Non-Functional Stories** (e.g., performance benchmarks, security, scalability).
            - Include priority, success criteria, and clear dependencies for each story.
            
            **Deliverables:**
            - Summarized total stories.
            - Key dependencies blocking progress."""
        ),
        llm=llm
    ),
    "Scrum_Master": Agent(
        name="Scrum_Master",
        system_prompt=(
            """You are the Scrum Master. Plan sprints efficiently and ensure all user stories are assigned with clear goals.
            
            **Key Tasks:**
            - Create a sprint plan with estimated velocity and clear tasks.
            - Provide a summary of sprint burndown expectations.
            
            **Deliverables:**
            - Number of sprints and their goal.
            - Short summary of team progress."""
        ),
        llm=llm
    ),
    "Requirement_Engineer": Agent(
        name="Requirement_Engineer",
        system_prompt=(
            """You are the Requirements Engineer. Write precise requirements for the Renter's Insurance platform.

            **Key Tasks:**
            - Functional Requirements: Clearly define each feature.
            - Non-Functional Requirements: Address system performance, usability, and scalability.
            - Data Needs: Highlight data requirements for system validation.
            
            **Deliverables:**
            - Detailed functional and non-functional requirements.
            - Testing data specifications."""
        ),
        llm=llm
    ),
    "System_Engineer": Agent(
        name="System_Engineer",
        system_prompt=(
            """You are the System Engineer. Develop a robust technical architecture.

            **Key Tasks:**
            - Create high-level diagrams showing interactions between modules.
            - Estimate SLOC for each module based on complexity.
            - Define APIs, database schemas, and integration protocols.

            **Deliverables:**
            - Architecture diagrams (include user authentication, claims, and payment flows).
            - Complexity classification (simple, medium, complex).
            - SLOC and dependencies for each module."""
        ),
        llm=llm
    ),
    "Software_Developer": Agent(
        name="Software_Developer",
        system_prompt=(
            """You are the Software Developer. Implement modular, efficient code for the Renter's Insurance platform.

            **Tasks for Each Module:**
            - Estimate SLOC.
            - Calculate development effort (SLOC / Productivity Rate; assume 50 SLOC/day).
            - Suggest optimizations for modularity and security.

            **Deliverables:**
            - SLOC estimates for modules.
            - Timeline showing start and end dates for each module."""
        ),
        llm=llm
    ),
    "Test_Engineer": Agent(
        name="Test_Engineer",
        system_prompt=(
            """You are the Test Engineer. Develop thorough test cases and automation strategies.

            **Key Tasks:**
            - Create functional and non-functional test cases.
            - Implement scalability and load testing strategies.
            - Log and categorize defects by severity.

            **Deliverables:**
            - Summary of test coverage.
            - Number of test cases for key features.
            - Metrics from load/performance testing."""
        ),
        llm=llm
    ),
    "Documentation_Engineer": Agent(
        name="Documentation_Engineer",
        system_prompt=(
            """You are the Documentation Engineer. Produce clear and organized documentation.

            **Key Tasks:**
            - Prepare technical documentation (APIs, architecture diagrams).
            - Create user guides for core functionalities.
            - Track sprint updates and lessons learned.

            **Deliverables:**
            - Version-controlled technical documents.
            - End-user manuals with illustrations."""
        ),
        llm=llm
    )
}


C:\Users\urjam\AppData\Local\Temp\ipykernel_2920\1193840116.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(


In [5]:
# Define prompts for specific activities
scrum_prompts = {
    "Sprint_Planning": {
        "Product_Owner": (
            "Prioritize and clarify details for {user_stories}. Identify dependencies and success criteria."
        ),
        "Requirement_Engineer": (
            "Define precise functional and non-functional requirements for {user_stories}."
        ),
        "System_Engineer": (
            "Design architecture for {user_stories}, highlighting key modules and complexity."
        ),
        "Software_Developer": (
            "Provide SLOC estimates and development timelines for {user_stories}."
        ),
        "Test_Engineer": (
            "Plan test cases and strategies for {user_stories} with focus on coverage."
        ),
        "Documentation_Engineer": (
            "Plan documentation updates for {user_stories}, ensuring clarity for end users."
        )
    }
}

In [6]:
# Function to run a sprint
def run_sprint(sprint_name, sprint_stories):
    print(f"--- {sprint_name} ---")
    print(f"User Stories: {', '.join(sprint_stories)}\n")

    for role, agent in agents.items():
        task_message = scrum_prompts["Sprint_Planning"].get(role, "").format(user_stories=", ".join(sprint_stories))
        response = agent.respond(task_message, "")
        print(f"{agent.name}:\n{response}\n")

In [7]:
# Example sprint execution
sprint_1_stories = ["User Registration and Secure Login"]
run_sprint("Sprint 1", sprint_1_stories)

--- Sprint 1 ---
User Stories: User Registration and Secure Login

Product_Owner:
### User Stories for User Registration and Secure Login

#### 1. User Registration (Functional Story)
- **As a** new user,  
- **I want** to register for an account on the Renter's Insurance platform,  
- **So that** I can access and manage my insurance policies.

**Acceptance Criteria:**
- Users must provide a valid email address, password, and agree to the terms and conditions.
- Users should receive a confirmation email upon successful registration.
- The system should validate the strength of the password.
- Email verification link must redirect users to the login page after confirming their email.

**Success Criteria:**
- At least 95% of users successfully complete registration.
- 90% of users successfully verify their email.
- Registration process takes no more than 5 minutes on average.

**Priority:** High  
**Dependencies:** 
- Email service for sending confirmation emails.
- User database setup t